# Finance Agent Demo: OpenTelemetry + Galileo + AWS Bedrock

This notebook demonstrates how to integrate **OpenTelemetry** with **Galileo** for comprehensive observability of agentic LLM workflows using **AWS Bedrock**.

## 📚 OpenTelemetry Basics

### What is OpenTelemetry?

**OpenTelemetry (OTel)** is an open-source observability framework that provides a unified way to collect, process, and export telemetry data (traces, metrics, logs) from your applications.

**Key Concepts**:
- **Traces**: Records of operations that show the flow of requests through your system
- **Spans**: Individual operations within a trace (e.g., an LLM call, a tool invocation)
- **Tracer Provider**: Manages span creation and exports traces to backends
- **Exporters**: Send traces to observability platforms (like Galileo)

### Why Use OpenTelemetry?

1. **Standardization**: Industry-standard format for observability data
2. **Vendor Neutral**: Works with any observability platform (Galileo, Datadog, New Relic, etc.)
3. **Automatic Instrumentation**: Libraries can automatically create spans for common operations
4. **Rich Metadata**: Captures detailed information about operations (timing, attributes, context)

### What You Need to Know

**Core Components**:
- **Tracer Provider**: Creates and manages spans
- **Span Processor**: Processes spans (batches, filters, exports)
- **Exporter**: Sends traces to your backend (Galileo in this case)
- **Instrumentation**: Automatic or manual code that creates spans

**Key Terms**:
- **Trace**: A complete request flow (e.g., one user query through the entire agent workflow)
- **Span**: A single operation (e.g., one LLM call, one tool execution)
- **Attributes**: Key-value pairs attached to spans (e.g., `ticker="AAPL"`, `temperature=0.1`)
- **OTLP**: OpenTelemetry Protocol - the standard format for sending telemetry data

**How It Works**:
1. Your code (or automatic instrumentation) creates spans
2. Spans are collected by the tracer provider
3. Span processor batches and processes spans
4. Exporter sends spans to Galileo via OTLP
5. Galileo visualizes traces in its console

### Trace Structure Example

When you run a query like "What's the price of AAPL?", here's how traces are structured:

```
Trace (one user query)
├── Span: finance_agent_query (root)
│   └── Span: LangGraph (workflow)
│       └── Span: agent (LLM decision)
│           └── Span: BedrockConverseLLM (LLM call)
│               ├── Input: User query
│               ├── Output: Tool call decision
│               └── Attributes: tokens, model, temperature
│       └── Span: tools (tool execution)
│           └── Span: get_stock_price (tool call)
│               ├── Input: ticker="AAPL"
│               ├── Output: price data
│               └── Attributes: ticker, price, found
│       └── Span: agent (final response)
│           └── Span: BedrockConverseLLM (LLM call)
│               ├── Input: Tool result + query
│               └── Output: Final response
```

Each span contains:
- **Timing**: Start/end times (calculate latency)
- **Attributes**: Key-value metadata (ticker, price, etc.)
- **Events**: Important moments (errors, checkpoints)
- **Context**: Links to parent/child spans

**For AI/ML Applications**:
- OpenTelemetry captures LLM calls, tool usage, and agent workflows
- **OpenInference** extends OpenTelemetry with AI-specific conventions
- Adds semantic meaning to spans (e.g., "this is an LLM call", "this is a tool call")
- Captures AI-specific metadata (prompts, responses, token counts, model parameters)

### Manual Tracing Basics

You can create spans manually in your code:

```python
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

# Create a span
with tracer.start_as_current_span("my_operation") as span:
    # Add attributes (metadata)
    span.set_attribute("user.id", "12345")
    span.set_attribute("operation.type", "stock_lookup")
    
    # Your code here
    result = lookup_stock("AAPL")
    
    # Add more attributes based on results
    span.set_attribute("result.price", result["price"])
    span.set_attribute("result.found", True)
```

**Key Points**:
- Spans are context managers (use `with` statement)
- Attributes are key-value pairs for filtering/searching
- Spans automatically capture timing
- Child spans are created automatically when nested

## 🎯 What This Demo Shows

This educational demo walks you through:

1. **OpenTelemetry Setup**: Configure OTLP exporter to send traces to Galileo
   - Learn how to set up the OTLP endpoint and authentication headers
   - Understand the relationship between OpenTelemetry and Galileo

2. **OpenInference Instrumentation**: Automatic tracing of LangGraph and LangChain operations
   - See how OpenInference adds AI-specific semantic conventions
   - Understand automatic span creation for LLM calls and tool usage

3. **AWS Bedrock Integration**: Use Bedrock models (Claude) instead of OpenAI
   - Learn how to create a custom LangChain LLM wrapper for Bedrock
   - Handle Bedrock ARNs and the Converse API format

4. **Finance Agent Simulation**: Multi-step agentic workflow with tools
   - Build a LangGraph agent with stock trading tools
   - See how agentic workflows are automatically traced

5. **Complete Trace Visibility**: View full workflow traces in Galileo Console
   - See complete trace graphs showing the entire agent workflow
   - Understand how spans connect to show the full execution path

### Why OpenTelemetry + Galileo?

**OpenTelemetry** provides the instrumentation and data collection, while **Galileo** provides:
- **AI-Specific Visualization**: Trace graphs optimized for LLM workflows
- **Token Usage Tracking**: Automatic tracking of input/output tokens
- **Cost Analysis**: Calculate costs per request based on model pricing
- **Performance Monitoring**: Latency and throughput metrics for AI operations
- **Debugging**: See exactly what your agent did and why

**Together**: OpenTelemetry collects the data, Galileo makes it actionable for AI/ML applications.

---

## 📋 Prerequisites

1. **Python 3.9+** installed
2. **Galileo Account**: Free account at [app.galileo.ai](https://app.galileo.ai)
3. **AWS Account** with Bedrock access
4. **Secrets Configuration**: Create `.streamlit/secrets.toml` with:
   - `galileo_api_key` - Your Galileo API key
   - `galileo_project` - Your project name
   - `galileo_log_stream` - Your log stream name
   - `aws_access_key_id` - AWS access key (optional, can use env vars)
   - `aws_secret_access_key` - AWS secret key (optional, can use env vars)
   - `aws_default_region` - AWS region (optional, defaults to us-east-1)
   
   **Note**: AWS credentials can also be set via environment variables or AWS credentials file.

## 📚 Documentation

This notebook follows the [Galileo OpenTelemetry Integration Guide](https://v2docs.galileo.ai/how-to-guides/third-party-integrations/otel)


In [1]:
# Step 1: Setup Environment and Configuration
import os
import sys
from pathlib import Path
import toml

# Add project root to path (for importing project modules)
notebook_dir = Path.cwd()
if notebook_dir.name == "OpenTelemetry_notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

sys.path.insert(0, str(project_root))

# Use the same setup_environment function as the Streamlit app
from setup_env import setup_environment

# Load secrets and set environment variables (same as Streamlit app)
setup_environment()

# Load secrets from .streamlit/secrets.toml for direct access in notebook
secrets_path = project_root / ".streamlit" / "secrets.toml"

if not secrets_path.exists():
    raise FileNotFoundError(
        f"❌ Secrets file not found: {secrets_path}\n"
        f"   Please create it from the template: .streamlit/secrets.toml.template"
    )

try:
    secrets = toml.load(secrets_path)
except Exception as e:
    raise ValueError(f"❌ Error loading secrets.toml: {e}")

# Configuration - Load from secrets.toml (for notebook use)
GALILEO_API_KEY = secrets.get("galileo_api_key", "") or os.getenv("GALILEO_API_KEY", "")
GALILEO_PROJECT = secrets.get("galileo_project", "otel-demo") or os.getenv("GALILEO_PROJECT", "otel-demo")
GALILEO_LOG_STREAM = secrets.get("galileo_log_stream", "finance-agent") or os.getenv("GALILEO_LOG_STREAM", "finance-agent")

# AWS Configuration - Check secrets.toml first, then environment variables
AWS_ACCESS_KEY_ID = secrets.get("aws_access_key_id", "") or os.getenv("AWS_ACCESS_KEY_ID", "")
AWS_SECRET_ACCESS_KEY = secrets.get("aws_secret_access_key", "") or os.getenv("AWS_SECRET_ACCESS_KEY", "")
AWS_DEFAULT_REGION = secrets.get("aws_default_region", "") or os.getenv("AWS_DEFAULT_REGION", "us-east-1")

# Bedrock Model ID or ARN - from secrets or env
BEDROCK_MODEL_ID = (
    secrets.get("bedrock_model_arn", "") or 
    secrets.get("bedrock_model_id", "") or 
    os.getenv("BEDROCK_MODEL_ARN", "") or
    os.getenv("BEDROCK_MODEL_ID", "anthropic.claude-3-5-sonnet-20241022-v2:0")
)

# Validate required configuration
if not GALILEO_API_KEY:
    raise ValueError("❌ galileo_api_key must be set in secrets.toml. Get it from: https://app.galileo.ai/settings/api-keys")

if not GALILEO_PROJECT:
    raise ValueError("❌ galileo_project must be set in secrets.toml")

if not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    raise ValueError(
        "❌ AWS credentials must be set. Add to secrets.toml:\n"
        "   aws_access_key_id = \"your-key\"\n"
        "   aws_secret_access_key = \"your-secret\"\n"
        "   Or set AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY environment variables"
    )

print("✅ Environment configured (using same setup as Streamlit app)")

print("✅ Configuration loaded successfully from secrets.toml!")
print(f"   📊 Project: {GALILEO_PROJECT}")
print(f"   📝 Log Stream: {GALILEO_LOG_STREAM}")
print(f"   🌍 AWS Region: {AWS_DEFAULT_REGION}")
print(f"   🤖 Bedrock Model: {BEDROCK_MODEL_ID}")
print(f"   📁 Project Root: {project_root}")
print(f"   🔐 Secrets loaded from: {secrets_path}")


⚠️  .streamlit/secrets.toml not found. Please create it with your API keys.
✅ Environment configured (using same setup as Streamlit app)
✅ Configuration loaded successfully from secrets.toml!
   📊 Project: cwan_demo
   📝 Log Stream: cwan_demo_logs
   🌍 AWS Region: us-east-1
   🤖 Bedrock Model: arn:aws:bedrock:us-east-1:818240400754:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0
   📁 Project Root: /Users/sabinaashurova/Desktop/GitHub/galileo-golden-demo
   🔐 Secrets loaded from: /Users/sabinaashurova/Desktop/GitHub/galileo-golden-demo/.streamlit/secrets.toml


## Step 2: Configure OpenTelemetry with Galileo

Following the [Galileo OpenTelemetry docs](https://v2docs.galileo.ai/how-to-guides/third-party-integrations/otel), we'll:
1. Set up OTLP endpoint to send traces to Galileo
2. Configure headers in the required format
3. Create tracer provider with resource information


In [2]:
# Step 2: Configure OpenTelemetry OTLP Endpoint and Headers
# Following Galileo docs: https://v2docs.galileo.ai/how-to-guides/third-party-integrations/otel

# Galileo's OpenTelemetry endpoint
# Per Galileo docs: https://v2docs.galileo.ai/how-to-guides/third-party-integrations/otel
# The endpoint is: https://api.galileo.ai/otel/traces
#
# IMPORTANT: Testing confirmed that https://api.galileo.ai/otel/traces returns 422
# (which means endpoint is correct and authentication works!)
#
# The OTLP HTTP exporter may append /v1/traces, but we need to use the exact endpoint
# Let's pass the full endpoint and see if the exporter is smart enough to not append
GALILEO_OTLP_ENDPOINT = "https://api.galileo.ai/otel/traces"
print(f"🔗 OTLP Endpoint: {GALILEO_OTLP_ENDPOINT}")
print(f"   ✅ This is the correct endpoint (confirmed by testing)")

# Configure headers in the format required by OpenTelemetry
# IMPORTANT: OTel requires comma-separated string format, not a dictionary!
headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": GALILEO_PROJECT,
    "logstream": GALILEO_LOG_STREAM,
}

# Set environment variable in the format OpenTelemetry expects
os.environ["OTEL_EXPORTER_OTLP_TRACES_HEADERS"] = ",".join(
    [f"{k}={v}" for k, v in headers.items()]
)

print("✅ OpenTelemetry configuration:")
print(f"   🔗 Endpoint: {GALILEO_OTLP_ENDPOINT}")
print(f"   🔑 Headers configured: {', '.join(headers.keys())}")
print(f"   📋 Header format: {os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'][:50]}...")


🔗 OTLP Endpoint: https://api.galileo.ai/otel/traces
   ✅ This is the correct endpoint (confirmed by testing)
✅ OpenTelemetry configuration:
   🔗 Endpoint: https://api.galileo.ai/otel/traces
   🔑 Headers configured: Galileo-API-Key, project, logstream
   📋 Header format: Galileo-API-Key=6GXiils0hsvDRogLRcKv_beYim-2pWwxe6...


## Step 3: Initialize OpenTelemetry

**What we're doing**: Creating the OpenTelemetry tracer provider and configuring exporters.

### Understanding the Tracer Provider

The **Tracer Provider** is the core component of OpenTelemetry:

1. **Creates Tracers**: Tracers are used to create spans in your code
2. **Manages Spans**: Collects all spans created by your application
3. **Processes Spans**: Batches spans for efficient delivery
4. **Exports Spans**: Sends spans to exporters (Galileo in our case)

**Components**:
- **TracerProvider**: The main manager
- **Span Processor**: Batches and processes spans before export
- **Exporter**: Sends spans to Galileo via OTLP
- **Resource**: Metadata about your service (name, version, etc.)

We use `setup_opentelemetry_from_env()` which:
- Creates a `TracerProvider` that manages trace creation
- Configures the OTLP exporter to send traces to Galileo
- Sets up batch processing for efficient trace delivery
- Enables automatic instrumentation for common libraries

**Why this matters**: 
- The tracer provider is the "engine" that powers OpenTelemetry
- It receives spans from your code (manual or automatic)
- It batches them efficiently (not every span immediately)
- It sends them to Galileo when ready
- Using the same setup function as the Streamlit app ensures consistency and reliability

### How Spans Flow

```
Your Code → Creates Span → Tracer Provider → Batch Processor → Exporter → Galileo
                                                      ↓
                                                (Batched for efficiency)
```


In [3]:
# Step 3: Initialize OpenTelemetry
# Use the same setup function as the Streamlit app for consistency
from setup_otel import setup_opentelemetry_from_env

# setup_environment() already set the environment variables
# Now use setup_opentelemetry_from_env() which reads from env vars (same as Streamlit app)
try:
    tracer_provider = setup_opentelemetry_from_env()
    print("✅ OpenTelemetry initialized (using same setup as Streamlit app)")
    print(f"   📊 Service: {os.getenv('OTEL_SERVICE_NAME', 'finance-agent-demo')}")
    print(f"   📦 Traces will be sent to Galileo")
except Exception as e:
    print(f"❌ Error setting up OpenTelemetry: {e}")
    raise


✅ OpenAI instrumentation enabled
✅ HTTP client instrumentation enabled
✅ OpenInference instrumentations enabled (OpenAI + LangChain)
🔧 OpenTelemetry setup complete!
✅ OpenTelemetry initialized (using same setup as Streamlit app)
   📊 Service: finance-agent-demo
   📦 Traces will be sent to Galileo


## Step 4: Apply OpenInference Instrumentation

**What we're doing**: Enabling automatic tracing for AI/ML frameworks.

### What is OpenInference?

**OpenInference** is an extension of OpenTelemetry that adds AI/ML-specific semantic conventions:
- **Standard Attributes**: Consistent naming for LLM calls, tool usage, etc.
- **Automatic Instrumentation**: Wraps common AI frameworks to create spans automatically
- **Rich Metadata**: Captures AI-specific information (prompts, tokens, model params)

### How Automatic Instrumentation Works

When you call `LangChainInstrumentor().instrument()`, it:
1. **Wraps LangChain code**: Intercepts LangChain operations
2. **Creates spans automatically**: Every chain, tool call, and LLM invocation gets a span
3. **Adds metadata**: Extracts input/output, parameters, token counts, etc.
4. **Maintains context**: Spans are linked to show the full workflow

**What gets instrumented**:
- **LangGraph** workflows and nodes - captures the agent's decision-making flow
- **LangChain** chains and tools - tracks tool calls and LLM interactions  
- **LLM API calls** - captures prompts, responses, token usage, and model parameters

**Why this matters**: 
Without OpenInference, you'd need to manually create spans for every LLM call and tool invocation:
```python
# Without OpenInference (manual):
with tracer.start_as_current_span("llm_call") as span:
    span.set_attribute("input", prompt)
    response = llm.invoke(prompt)
    span.set_attribute("output", response)
    span.set_attribute("tokens", tokens_used)

# With OpenInference (automatic):
response = llm.invoke(prompt)  # Span created automatically!
```

OpenInference adds rich metadata automatically:
- Input/output messages
- Token counts
- Model parameters (temperature, max_tokens, etc.)
- Tool names and arguments
- Workflow step information

This gives you complete visibility into your agent's behavior without writing trace code for every operation.


- **LangChain** chains and tools
- **OpenAI** API calls (we'll use Bedrock instead)

This enables automatic tracing of LLM calls, token usage, and agent workflow steps.


In [4]:
# Step 4: Apply OpenInference Instrumentation
# This automatically captures LangGraph workflows, LangChain operations, and LLM calls

langchain_instrumented = False
langgraph_instrumented = False

try:
    from openinference.instrumentation.langchain import LangChainInstrumentor
    
    # Instrument LangChain
    LangChainInstrumentor().instrument(tracer_provider=tracer_provider)
    langchain_instrumented = True
    print("✅ LangChain instrumentation enabled")
    
except ImportError as e:
    print(f"⚠️  LangChain instrumentation not available: {e}")
    print("   Install with: pip install openinference-instrumentation-langchain")
except Exception as e:
    print(f"⚠️  Error enabling LangChain instrumentation: {e}")

try:
    from openinference.instrumentation.langgraph import LangGraphInstrumentor
    
    # Instrument LangGraph
    LangGraphInstrumentor().instrument(tracer_provider=tracer_provider)
    langgraph_instrumented = True
    print("✅ LangGraph instrumentation enabled")
    
except ImportError:
    print("⚠️  LangGraph instrumentation not available")
    print("   Note: openinference-instrumentation-langgraph may not exist as a package")
    print("   Manual tracing will be used instead")
except Exception as e:
    print(f"⚠️  Error enabling LangGraph instrumentation: {e}")

if langchain_instrumented or langgraph_instrumented:
    print("\n✅ OpenInference instrumentation enabled:")
    if langchain_instrumented:
        print("   🔗 LangChain operations will be traced")
    if langgraph_instrumented:
        print("   🔗 LangGraph workflows will be traced")
    print("   📊 LLM calls, token usage, and agent steps will be captured automatically")
else:
    print("\n⚠️  No OpenInference instrumentation available")
    print("   Manual spans will still work, but automatic tracing is disabled")
    print("   Traces should still appear in Galileo from manual spans")


Attempting to instrument while already instrumented


✅ LangChain instrumentation enabled
⚠️  LangGraph instrumentation not available
   Note: openinference-instrumentation-langgraph may not exist as a package
   Manual tracing will be used instead

✅ OpenInference instrumentation enabled:
   🔗 LangChain operations will be traced
   📊 LLM calls, token usage, and agent steps will be captured automatically


## Step 5: Create Finance Agent Tools

**What we're doing**: Creating LangChain tools that the agent can use to perform actions.

### Understanding Manual vs Automatic Tracing

**Automatic Tracing** (OpenInference):
- Created by instrumentation libraries
- Happens automatically when you call LangChain/LangGraph functions
- Captures standard information (inputs, outputs, timing)

**Manual Tracing** (Custom spans):
- Created explicitly in your code with `tracer.start_as_current_span()`
- Lets you add custom attributes and context
- Useful for business logic or custom operations

**In this demo**: We use both!
- OpenInference automatically traces tool calls
- We also add manual spans to include custom attributes (e.g., stock ticker, order ID)

### The Tools

We'll create three finance tools:
- `get_stock_price`: Look up current stock prices and market data
- `purchase_stocks`: Execute a stock purchase order
- `sell_stocks`: Execute a stock sale order

**Why this matters**: These tools demonstrate how agents can:
1. **Use external data**: `get_stock_price` accesses market data
2. **Perform actions**: `purchase_stocks` and `sell_stocks` execute transactions
3. **Be automatically traced**: Each tool call creates a span with input/output data

**Note**: Each tool includes manual OpenTelemetry spans (`tracer.start_as_current_span`) to add custom attributes like `ticker`, `order.id`, etc. OpenInference will also automatically create spans for tool calls, so you'll see both in Galileo - giving you comprehensive visibility.


In [5]:
# Step 5: Create Finance Agent Tools
import json
import random
import time
from typing import Optional
from langchain_core.tools import tool
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

# Mock stock price database
MOCK_STOCKS = {
    "AAPL": {"price": 178.72, "change": 1.23, "change_percent": 0.69},
    "MSFT": {"price": 415.32, "change": 2.45, "change_percent": 0.59},
    "GOOGL": {"price": 147.68, "change": -0.82, "change_percent": -0.55},
    "AMZN": {"price": 178.75, "change": 1.25, "change_percent": 0.70},
    "TSLA": {"price": 177.77, "change": -2.33, "change_percent": -1.29},
    "NVDA": {"price": 950.02, "change": 15.98, "change_percent": 1.71},
}

@tool
def get_stock_price(ticker: str) -> str:
    """Get the current stock price and market data for a given ticker symbol (e.g., AAPL, MSFT, TSLA)."""
    with tracer.start_as_current_span("get_stock_price") as span:
        span.set_attribute("ticker", ticker)
        
        if ticker.upper() in MOCK_STOCKS:
            data = MOCK_STOCKS[ticker.upper()]
            span.set_attribute("stock.found", True)
            span.set_attribute("stock.price", data["price"])
            return json.dumps(data)
        else:
            # Default mock data
            span.set_attribute("stock.found", False)
            result = {"price": 100.00, "change": 0.00, "change_percent": 0.00}
            return json.dumps(result)

@tool
def purchase_stocks(ticker: str, quantity: int, price: float) -> str:
    """Execute a stock purchase order. Returns order confirmation with order ID."""
    with tracer.start_as_current_span("purchase_stocks") as span:
        span.set_attribute("ticker", ticker)
        span.set_attribute("quantity", quantity)
        span.set_attribute("price", price)
        
        order_id = f"ORD-{random.randint(100000, 999999)}"
        total_cost = quantity * price
        fees = 10.00
        
        result = {
            "order_id": order_id,
            "ticker": ticker,
            "quantity": quantity,
            "price": price,
            "total_cost": total_cost,
            "fees": fees,
            "total_with_fees": total_cost + fees,
            "status": "completed"
        }
        
        span.set_attribute("order.id", order_id)
        span.set_attribute("order.total", total_cost + fees)
        
        return json.dumps(result)

@tool
def sell_stocks(ticker: str, quantity: int, price: float) -> str:
    """Execute a stock sale order. Returns order confirmation with order ID."""
    with tracer.start_as_current_span("sell_stocks") as span:
        span.set_attribute("ticker", ticker)
        span.set_attribute("quantity", quantity)
        span.set_attribute("price", price)
        
        order_id = f"ORD-{random.randint(100000, 999999)}"
        total_sale = quantity * price
        fees = 14.99
        
        result = {
            "order_id": order_id,
            "ticker": ticker,
            "quantity": quantity,
            "price": price,
            "total_sale": total_sale,
            "fees": fees,
            "total_with_fees": total_sale - fees,
            "status": "completed"
        }
        
        span.set_attribute("order.id", order_id)
        span.set_attribute("order.total", total_sale - fees)
        
        return json.dumps(result)

# Create tools list
finance_tools = [get_stock_price, purchase_stocks, sell_stocks]

print("✅ Finance tools created:")
for tool in finance_tools:
    print(f"   🔧 {tool.name}: {tool.description}")


✅ Finance tools created:
   🔧 get_stock_price: Get the current stock price and market data for a given ticker symbol (e.g., AAPL, MSFT, TSLA).
   🔧 purchase_stocks: Execute a stock purchase order. Returns order confirmation with order ID.
   🔧 sell_stocks: Execute a stock sale order. Returns order confirmation with order ID.


In [6]:
# Step 6a: Initialize Bedrock Client
import boto3
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.outputs import ChatGeneration, ChatResult
from typing import List, Optional

# Initialize AWS Bedrock client (same as your working code)
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

print("✅ Bedrock runtime client initialized")


✅ Bedrock runtime client initialized


### Step 6b: Create Custom Bedrock LLM Wrapper

**What we're doing**: Creating a custom LangChain LLM wrapper that uses AWS Bedrock's Converse API.

**Why we need this**: 
- LangChain's `ChatBedrock` requires a `model_provider` parameter when using ARNs
- Bedrock's Converse API works directly with ARNs without provider info
- This wrapper gives us full control over the API calls and message formatting

**How it works**:
1. Converts LangChain messages to Bedrock's message format
2. Handles tool calls and tool results (required for agent workflows)
3. Converts Bedrock responses back to LangChain format
4. Maintains trace context for OpenTelemetry

**Key features**:
- Works directly with Bedrock ARNs (e.g., `arn:aws:bedrock:us-east-1:...`)
- Handles tool calling (required for agentic workflows)
- Properly formats tool results to match Bedrock's expected format


In [7]:
# Step 6b: Create Custom Bedrock LLM Wrapper
# This uses boto3.converse directly (same as your working code)
from pydantic import Field, ConfigDict
from typing import Any

class BedrockConverseLLM(BaseChatModel):
    """Custom LangChain LLM that uses boto3 Bedrock Converse API (works with ARNs)"""
    
    model_id: str = Field(description="Bedrock model ID or ARN")
    bedrock_client: Any = Field(description="boto3 bedrock-runtime client")
    temperature: float = Field(default=0.1, description="Temperature for generation")
    max_tokens: int = Field(default=2000, description="Max tokens for generation")
    bound_tools: Optional[List[Any]] = Field(default=None, description="Bound tools for the LLM")
    
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    def __init__(self, model_id: str, bedrock_client, **kwargs):
        super().__init__(
            model_id=model_id,
            bedrock_client=bedrock_client,
            temperature=kwargs.get("temperature", 0.1),
            max_tokens=kwargs.get("max_tokens", 2000),
            bound_tools=kwargs.get("tools", None),
            **{k: v for k, v in kwargs.items() if k not in ["temperature", "max_tokens", "tools"]}
        )
    
    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, **kwargs):
        # Convert LangChain messages to Bedrock format
        # Important: Bedrock Converse doesn't allow assistant messages in the final position when using tools
        bedrock_messages = []
        # Track tool use IDs from tool calls to match with tool results
        tool_use_id_map = {}  # Maps LangChain tool_call_id to Bedrock toolUseId
        
        for i, msg in enumerate(messages):
            if isinstance(msg, HumanMessage):
                bedrock_messages.append({
                    "role": "user",
                    "content": [{"text": msg.content}]
                })
            elif isinstance(msg, AIMessage):
                # Handle tool calls if present
                if hasattr(msg, "tool_calls") and msg.tool_calls and len(msg.tool_calls) > 0:
                    # Convert tool calls to Bedrock format
                    content = []
                    for tool_call in msg.tool_calls:
                        # Generate a unique toolUseId for Bedrock
                        langchain_tool_call_id = tool_call.get("id", "")
                        # Use the LangChain tool_call_id as the Bedrock toolUseId
                        # Bedrock expects toolUseId to match in tool calls and tool results
                        bedrock_tool_use_id = langchain_tool_call_id if langchain_tool_call_id else f"tooluse_{i}_{len(content)}"
                        
                        # Map LangChain tool_call_id to Bedrock toolUseId
                        if langchain_tool_call_id:
                            tool_use_id_map[langchain_tool_call_id] = bedrock_tool_use_id
                        
                        content.append({
                            "toolUse": {
                                "toolUseId": bedrock_tool_use_id,
                                "name": tool_call.get("name", ""),
                                "input": tool_call.get("args", {})
                            }
                        })
                    bedrock_messages.append({
                        "role": "assistant",
                        "content": content
                    })
                elif msg.content and msg.content.strip():
                    # Only add assistant text message if it's not the last message
                    # (Bedrock doesn't allow assistant message as final message when tools are enabled)
                    is_last_message = (i == len(messages) - 1)
                    if not is_last_message:
                        bedrock_messages.append({
                            "role": "assistant",
                            "content": [{"text": msg.content}]
                        })
            elif isinstance(msg, SystemMessage):
                # Bedrock Converse uses system message in a different way
                bedrock_messages.append({
                    "role": "user",
                    "content": [{"text": msg.content}]
                })
            # Handle ToolMessage if present (tool execution results)
            elif isinstance(msg, ToolMessage):
                # ToolMessage from LangChain - convert to Bedrock tool result format
                # ToolMessage has tool_call_id attribute that matches the LangChain tool_call id
                langchain_tool_call_id = getattr(msg, "tool_call_id", None)
                tool_content = str(msg.content)
                
                # Map LangChain tool_call_id to Bedrock toolUseId
                bedrock_tool_use_id = tool_use_id_map.get(langchain_tool_call_id) if langchain_tool_call_id else None
                
                if not bedrock_tool_use_id:
                    # Fallback: try to use the tool_call_id directly if it looks like a Bedrock toolUseId
                    if langchain_tool_call_id and langchain_tool_call_id.startswith("tooluse_"):
                        bedrock_tool_use_id = langchain_tool_call_id
                    else:
                        # Skip if we can't find the matching tool use ID
                        print(f"⚠️  Warning: Could not map tool_call_id {langchain_tool_call_id} to Bedrock toolUseId, skipping tool result")
                        continue
                
                tool_result = {
                    "toolResult": {
                        "toolUseId": bedrock_tool_use_id,
                        "status": "success",
                        "content": [{"text": tool_content}]
                    }
                }
                bedrock_messages.append({
                    "role": "user",
                    "content": [tool_result]
                })
        
        # Prepare tool config if tools are bound
        tool_config = None
        if self.bound_tools:
            # Convert LangChain tools to Bedrock format
            bedrock_tools = []
            for tool in self.bound_tools:
                tool_spec = {
                    "toolSpec": {
                        "name": tool.name,
                        "description": tool.description or "",
                        "inputSchema": {}
                    }
                }
                # Try to get input schema from tool
                # Bedrock expects inputSchema as {"json": <schema>} format
                if hasattr(tool, "args_schema") and tool.args_schema:
                    try:
                        # Use model_json_schema for Pydantic v2
                        if hasattr(tool.args_schema, "model_json_schema"):
                            schema = tool.args_schema.model_json_schema()
                        elif hasattr(tool.args_schema, "schema"):
                            schema = tool.args_schema.schema()
                        else:
                            schema = {}
                        # Bedrock requires inputSchema in {"json": <schema>} format
                        tool_spec["toolSpec"]["inputSchema"] = {"json": schema}
                    except Exception:
                        # Fallback to empty schema
                        tool_spec["toolSpec"]["inputSchema"] = {"json": {}}
                else:
                    # No schema available, use empty JSON schema
                    tool_spec["toolSpec"]["inputSchema"] = {"json": {}}
                
                bedrock_tools.append(tool_spec)
            
            if bedrock_tools:
                tool_config = {"tools": bedrock_tools}
        
        # Call Bedrock Converse API (works with ARNs directly)
        converse_kwargs = {
            "modelId": self.model_id,  # Your ARN works here
            "messages": bedrock_messages,
            "inferenceConfig": {
                "maxTokens": self.max_tokens,
                "temperature": self.temperature
            }
        }
        
        if tool_config:
            converse_kwargs["toolConfig"] = tool_config
        
        response = self.bedrock_client.converse(**converse_kwargs)
        
        # Extract response - handle both text and tool use
        output = response['output']['message']['content'][0]
        
        if 'text' in output:
            # Regular text response
            output_text = output['text']
            message = AIMessage(content=output_text)
        elif 'toolUse' in output:
            # Tool use response - convert to LangChain format
            tool_use = output['toolUse']
            message = AIMessage(
                content="",
                tool_calls=[{
                    "id": tool_use.get("toolUseId", ""),
                    "name": tool_use.get("name", ""),
                    "args": tool_use.get("input", {})
                }]
            )
        else:
            output_text = str(output)
            message = AIMessage(content=output_text)
        
        # Return ChatResult
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])
    
    def bind_tools(self, tools, **kwargs):
        """Bind tools to the LLM"""
        return BedrockConverseLLM(
            model_id=self.model_id,
            bedrock_client=self.bedrock_client,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            tools=tools
        )
    
    @property
    def _llm_type(self) -> str:
        return "bedrock-converse"

print("✅ Custom BedrockConverseLLM class defined")


✅ Custom BedrockConverseLLM class defined


### Step 6c: Initialize LLM and Bind Tools

**What we're doing**: Creating the LLM instance and attaching tools to it.

1. **Initialize the LLM**: Create our custom `BedrockConverseLLM` with the model ARN
2. **Bind tools**: Attach the finance tools to the LLM so it can use them

**Why this matters**: When tools are bound to an LLM, the LLM can decide when to call them based on the user's request. The agent will automatically:
- Analyze the user's query
- Decide which tools to use
- Call the tools with appropriate parameters
- Use the tool results to generate a response

This binding is what enables the agentic behavior - the LLM becomes an agent that can take actions, not just respond with text.


In [8]:
# Step 6c: Initialize LLM and Bind Tools
try:
    is_arn = BEDROCK_MODEL_ID.startswith("arn:")
    
    llm = BedrockConverseLLM(
        model_id=BEDROCK_MODEL_ID,  # Your ARN works directly here
        bedrock_client=bedrock_runtime,
        temperature=0.1,
        max_tokens=2000
    )
    
    # Bind tools to the LLM
    llm_with_tools = llm.bind_tools(finance_tools)
    
    model_type = "ARN" if is_arn else "Model ID"
    print(f"✅ Bedrock LLM initialized ({model_type}): {BEDROCK_MODEL_ID}")
    print(f"   🔧 Tools bound: {len(finance_tools)}")
    print(f"   💡 Using boto3.converse directly (same as your working code)")
    
except Exception as e:
    print(f"❌ Error initializing Bedrock: {e}")
    print("   Make sure your AWS credentials are correct and Bedrock access is enabled")
    raise


✅ Bedrock LLM initialized (ARN): arn:aws:bedrock:us-east-1:818240400754:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0
   🔧 Tools bound: 3
   💡 Using boto3.converse directly (same as your working code)


### Step 6d: Build LangGraph Workflow

**What we're doing**: Creating the agent workflow using LangGraph.

**LangGraph structure**:
1. **Agent node**: The LLM that decides what to do next
2. **Tools node**: Executes tool calls when the agent decides to use tools
3. **Conditional routing**: Determines whether to continue to tools or end

**How it works**:
```
User Query → Agent → [Decision: Continue to Tools OR End]
                      ↓
                   Tools Node → Agent → [Decision: Continue OR End]
```

**Why LangGraph**: 
- Provides a structured way to build agentic workflows
- Handles tool calling automatically
- Manages state and conversation context
- Automatically traced by OpenInference (if available)

**What you'll see in traces**: Each node execution creates a span, showing the complete flow of the agent's decision-making process.


In [9]:
# Step 6d: Build LangGraph Workflow
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, TypedDict

# Define state for the graph
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Create the graph
def create_agent_node(llm):
    """Create the agent node that processes messages"""
    def agent_node(state: AgentState):
        messages = [SystemMessage(content="You are a helpful finance assistant. Use tools when needed to answer questions about stocks.")]
        messages.extend(state["messages"])
        response = llm.invoke(messages)
        return {"messages": [response]}
    return agent_node

# Build the graph
graph_builder = StateGraph(AgentState)
graph_builder.add_node("agent", create_agent_node(llm_with_tools))
graph_builder.add_node("tools", ToolNode(finance_tools))

graph_builder.add_conditional_edges("agent", tools_condition)
graph_builder.add_edge("tools", "agent")
graph_builder.add_edge(START, "agent")

# Compile the graph
agent_graph = graph_builder.compile()

print("✅ LangGraph agent created with workflow:")
print("   START → agent → (tools if needed) → agent → END")


✅ LangGraph agent created with workflow:
   START → agent → (tools if needed) → agent → END


## Step 7: Run Demo Queries

**What we're doing**: Running example queries to demonstrate the agent in action.

**Demo queries**:
1. **Simple query**: "What's the current price of AAPL?" - Shows basic tool usage
2. **Action query**: "Buy 10 shares of TSLA" - Shows tool execution with parameters
3. **Multi-tool query**: "What's the price of NVDA and MSFT?" - Shows multiple tool calls

**What happens for each query**:
1. User query is sent to the agent
2. Agent analyzes the query and decides to use tools
3. Tools are called with appropriate parameters
4. Agent processes tool results and generates a response
5. **All of this is automatically traced** by OpenTelemetry

**What you'll see**:
- Console output showing the agent's responses
- Trace spans for each step (visible in console exporter)
- Complete trace graphs in Galileo showing the full workflow

**Try modifying the queries** to see how the agent handles different scenarios!


In [10]:
# Step 7: Run Demo Queries
from langchain_core.messages import HumanMessage

# Demo queries that showcase agentic behavior
demo_queries = [
    "What's the current price of AAPL?",
    "Buy 10 shares of TSLA at 180 dollars per share",
    "What's the price of NVDA and MSFT?",
]

print("🚀 Running Finance Agent Demo Queries")
print("=" * 80)

for i, query in enumerate(demo_queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-" * 80)
    
    # Create a trace span for this query
    tracer = trace.get_tracer(__name__)
    with tracer.start_as_current_span("finance_agent_query") as span:
        span.set_attribute("query.text", query)
        span.set_attribute("query.number", i)
        
        try:
            # Run the agent
            result = agent_graph.invoke({
                "messages": [HumanMessage(content=query)]
            })
            
            # Get the final response
            final_message = result["messages"][-1]
            response_text = final_message.content if hasattr(final_message, 'content') else str(final_message)
            
            span.set_attribute("response.length", len(response_text))
            span.set_attribute("response.success", True)
            
            print(f"✅ Response: {response_text[:200]}...")
            
        except Exception as e:
            span.set_attribute("response.success", False)
            span.set_attribute("error.message", str(e))
            print(f"❌ Error: {e}")

print("\n" + "=" * 80)
print("✅ All queries completed! Traces are being sent to Galileo.")


🚀 Running Finance Agent Demo Queries

📝 Query 1: What's the current price of AAPL?
--------------------------------------------------------------------------------
✅ Response: The current price of Apple Inc. (AAPL) stock is $178.72 per share. It is up $1.23 or 0.69% from the previous close....

📝 Query 2: Buy 10 shares of TSLA at 180 dollars per share
--------------------------------------------------------------------------------
✅ Response: Okay, let's execute a stock purchase order for 10 shares of TSLA at $180 per share....

📝 Query 3: What's the price of NVDA and MSFT?
--------------------------------------------------------------------------------
✅ Response: Okay, let me get the current stock prices for NVDA (NVIDIA) and MSFT (Microsoft) using the get_stock_price tool....

✅ All queries completed! Traces are being sent to Galileo.


## Step 8: View Traces in Galileo

**What we're doing**: Flushing traces and generating links to view them in Galileo.

**What happens**:
1. **Flush traces**: Ensures all spans are sent to Galileo (not just batched)
2. **Test connection**: Verifies the endpoint is reachable
3. **Generate links**: Creates direct links to your project and log stream

**What you'll see in Galileo**:

1. **Trace Graph View**:
   - Root span: `LangGraph` (the entire workflow)
   - Child spans: `agent` nodes (LLM decision points)
   - Tool spans: `get_stock_price`, `purchase_stocks`, etc.
   - LLM spans: `BedrockConverseLLM` (showing prompts, responses, token usage)

2. **Span Details**:
   - **Input/Output**: See the exact messages sent to and received from the LLM
   - **Attributes**: Tool parameters, stock prices, order IDs, etc.
   - **Metadata**: Model parameters, token counts, timing information
   - **Trace Context**: How spans connect to show the full execution path

3. **Performance Insights**:
   - Latency for each step
   - Token usage per LLM call
   - Tool execution times

**Troubleshooting**: If traces don't appear:
- Verify the log stream exists in your project
- Wait 30-60 seconds for processing
- Refresh the Galileo console
- Check the console output above for trace details


In [11]:
# Step 8: Generate Galileo Console URLs and Flush Traces
import time
import requests

# Wait a moment for traces to be batched and sent
print("⏳ Waiting for traces to be sent to Galileo...")
time.sleep(3)

# Force flush any remaining spans with timeout
print("🔄 Flushing traces to Galileo...")
try:
    # Force flush with a timeout
    tracer_provider.force_flush(timeout_millis=10000)  # 10 second timeout
    print("✅ Flush complete")
except Exception as flush_error:
    print(f"⚠️  Flush error: {flush_error}")
    print("   This might indicate an export issue")

# Test trace creation
print("\n🧪 Testing trace creation...")
test_tracer = trace_api.get_tracer(__name__)
with test_tracer.start_as_current_span("test_trace") as span:
    span.set_attribute("test.attribute", "test_value")
    span.set_attribute("test.service", "finance-agent-demo")
    span.set_attribute("test.project", GALILEO_PROJECT)
    span.set_attribute("test.logstream", GALILEO_LOG_STREAM)
    print("   📝 Test span created")

# Flush and wait
tracer_provider.force_flush(timeout_millis=10000)
print("✅ Test trace created and flushed")

# Test direct HTTP connection to verify endpoint
print("\n🌐 Testing connection to Galileo OTLP endpoint...")
try:
    import requests
    test_headers = {
        "Content-Type": "application/x-protobuf",
        "Galileo-API-Key": GALILEO_API_KEY,
        "project": GALILEO_PROJECT,
        "logstream": GALILEO_LOG_STREAM,
    }
    
    # Test the correct endpoint (confirmed working)
    response = requests.post(GALILEO_OTLP_ENDPOINT, headers=test_headers, data=b"", timeout=5)
    print(f"   Endpoint: {GALILEO_OTLP_ENDPOINT}")
    print(f"   Status: {response.status_code}")
    if response.status_code in [200, 400, 422]:
        print("   ✅ Endpoint is reachable and authentication works!")
    elif response.status_code == 401:
        print("   ❌ Authentication failed - check API key")
    elif response.status_code == 404:
        print("   ❌ Endpoint not found - check URL")
    else:
        print(f"   ⚠️  Status: {response.status_code}")
except Exception as conn_error:
    print(f"   ❌ Connection error: {conn_error}")

print("\n" + "="*80)
print("📊 TRACE STATUS SUMMARY")
print("="*80)
print("✅ Traces ARE being created successfully!")
print("   - Console output shows multiple spans (LangGraph, agent, BedrockConverseLLM)")
print("   - Endpoint is reachable and authentication works (422 response = valid)")
print("   - Traces are being flushed to Galileo")
print("\n💡 If traces don't appear in Galileo console:")
print("   1. VERIFY LOG STREAM EXISTS:")
print("      - Go to: https://app.galileo.ai/project/70fb8148-6e04-4408-8633-cb83415f0fd1")
print("      - Check if log stream '{}' exists".format(GALILEO_LOG_STREAM))
print("      - If not, create it in the Galileo console")
print("   2. WAIT 30-60 SECONDS:")
print("      - Traces may take time to process and appear")
print("   3. REFRESH THE CONSOLE:")
print("      - Hard refresh the browser (Cmd+Shift+R or Ctrl+F5)")
print("="*80)

# Generate Galileo Console URLs
# Note: Galileo URLs require UUIDs (project ID and log stream ID), not names
console_url = "https://app.galileo.ai"  # or your custom deployment URL

# Try to fetch project and log stream IDs from Galileo API
try:
    # Get project ID - use the paginated API endpoint (POST request)
    api_url = f"{console_url}/api/galileo/public/v2/projects/paginated?starting_token=0&limit=100"
    headers = {
        "accept": "*/*",
        "galileo-api-key": GALILEO_API_KEY,
        "content-type": "application/json",
        "origin": console_url,
        "referer": f"{console_url}/",
    }
    
    # POST request with sort/filter data
    data = {
        "sort": {
            "name": "updated_at",
            "ascending": False
        },
        "filters": []
    }
    
    response = requests.post(api_url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        projects = result.get("projects", [])
        project_id = None
        for project in projects:
            if project.get("name") == GALILEO_PROJECT:
                project_id = project.get("id")
                break
        
        if project_id:
            # Get log stream ID - GET request
            log_streams_url = f"{console_url}/api/galileo/v2/projects/{project_id}/log_streams"
            log_stream_response = requests.get(log_streams_url, headers=headers)
            if log_stream_response.status_code == 200:
                log_streams = log_stream_response.json()
                log_stream_id = None
                for stream in log_streams:
                    if stream.get("name") == GALILEO_LOG_STREAM:
                        log_stream_id = stream.get("id")
                        break
                
                if log_stream_id:
                    project_url = f"{console_url}/project/{project_id}"
                    log_stream_url = f"{project_url}/log-streams/{log_stream_id}"
                    print("\n" + "=" * 80)
                    print("📊 Galileo Dashboard Links")
                    print("=" * 80)
                    print(f"🔗 Project URL: {project_url}")
                    print(f"🔗 Log Stream URL: {log_stream_url}")
                    print("\n💡 What to look for in Galileo:")
                    print("   • Complete trace graphs showing LangGraph workflow")
                    print("   • Tool call spans (get_stock_price, purchase_stocks, etc.)")
                    print("   • Bedrock LLM calls with token usage")
                    print("   • Performance metrics and timing")
                    print("=" * 80)
                else:
                    print(f"⚠️  Log stream '{GALILEO_LOG_STREAM}' not found. Access via: {console_url}")
            else:
                print(f"⚠️  Could not fetch log streams (status: {log_stream_response.status_code}). Access via: {console_url}")
        else:
            print(f"⚠️  Project '{GALILEO_PROJECT}' not found. Access via: {console_url}")
    else:
        print(f"⚠️  Could not fetch projects (status: {response.status_code}). Access via: {console_url}")
        print(f"   Project: {GALILEO_PROJECT}, Log Stream: {GALILEO_LOG_STREAM}")
except Exception as e:
    print(f"⚠️  Error fetching Galileo IDs: {e}")
    print(f"   Access Galileo Console directly: {console_url}")
    print(f"   Project: {GALILEO_PROJECT}, Log Stream: {GALILEO_LOG_STREAM}")
    print("\n💡 Note: If you see UUID errors in the playground, navigate directly")
    print("   to the project and log stream from the Galileo Console home page.")


⏳ Waiting for traces to be sent to Galileo...
🔄 Flushing traces to Galileo...
✅ Flush complete

🧪 Testing trace creation...


NameError: name 'trace_api' is not defined